In [1]:
from tensorflow.keras.utils import Sequence
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from tensorflow.keras.layers import Concatenate,Embedding ,Dense ,Input,LSTM,Permute,Softmax,Lambda,Flatten,GRU,Dropout,BatchNormalization, Normalization, Attention, Bidirectional, Masking
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.metrics import F1Score, Accuracy
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
#!pip install tensorflow-addons
#import tensorflow_addons as tfa
from tensorflow.keras.losses import BinaryCrossentropy
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import h5py
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

C:\Users\homan\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class DataGenerator(Sequence):

    def __init__(self, meta_df, batch_size=64, use_full=False, input_shape=(20,196), **kwargs):
        super().__init__(**kwargs)
        self.ds_pointer = 0
        self.batch_size = batch_size
        self.use_full = use_full
        self.meta_df = meta_df
        self.data = self.load_file()
        self.input_buffer = np.zeros((batch_size,input_shape[0],input_shape[1]))
        self.label_buffer = np.zeros((batch_size,1))
        self.reset_pointer()

    def load_file(self):
        result = {}
        for root, _, files in os.walk('data/train_data'):
            for file in files:
                if f'.h5' in file:
                    symbol = file.split('.')[0]
                    df = pd.read_csv(f'data/price/{symbol}.csv')
                    result[symbol] = h5py.File(f'data/train_data/{symbol}.h5', 'r', rdcc_nbytes=100*1024**2, rdcc_nslots=1e4)
        return result

    def reset_pointer(self, seed = 1314):
        self.meta_df = self.meta_df.sample(frac=1, random_state=seed)
        self.ds_pointer = 0

    def __len__(self):
        if self.use_full:
            return len(self.meta_df)// self.batch_size
        else:
            return len(self.meta_df)// self.batch_size// 10

    def num_labels(self):
        return len(self.label_name)

    def __getitem__(self, index):
        import os
        import numpy as np
        if self.ds_pointer + self.batch_size >= len(self.meta_df):
            self.reset_pointer()
            return self.__getitem__(index)
        to_get = self.meta_df.iloc[self.ds_pointer:self.ds_pointer+ self.batch_size]
        to_get_index = to_get['Index'].to_numpy()
        to_get_symbol = to_get['Symbol'].to_numpy()

        for i, symbol in enumerate(to_get_symbol):
            to_fetch = self.data[symbol]
            self.input_buffer[i, :, :] = to_fetch['data'][to_get_index[i]]
            self.label_buffer[i] = to_fetch['label'][to_get_index[i]]
            
        batch_x = self.input_buffer
        batch_y = self.label_buffer.reshape(-1,1)
        
        self.ds_pointer+=self.batch_size
        return batch_x, batch_y

In [40]:

def build_model(time_series,num_of_features):
    K.clear_session()
    input_ = Input(shape=(time_series,num_of_features), name='Input')
    input_dense = Dense(256)(input_)
    # Define GRU layer
    encoder = GRU(256, return_state=True, return_sequences=True, name='Encoder')
    encoder_outputs, state_h = encoder(input_dense)

    # # Define attention layer
    attention = Attention(name='Attention')
    context_vector = attention([encoder_outputs, state_h])

    # Concatenate context vector and encoder outputs
    concat_layer = Concatenate(axis=-1, name='Concatenate')
    decoder_combined_context = concat_layer([context_vector, input_dense])

    # Define decoder
    decoder_gru = GRU(256, return_sequences=True, name='Decoder')

    # Pass the concatenated input through the decoder
    decoder_output = decoder_gru(decoder_combined_context, initial_state=state_h)

    output = Flatten(name='Flatten')(decoder_output)
    output = Dropout(0.5)(output)
    output = BatchNormalization()(output)
    output = Dense(256,activation="relu")(output)
    output = Dropout(0.5)(output)
    output = BatchNormalization()(output)
    output = Dense(128, activation='relu')(output)
    output = Dropout(0.5)(output)
    output = BatchNormalization()(output)
    output = Dense(1, activation='sigmoid')(output)


    model = Model(inputs = input_ , outputs = output)
    adam_optimizer = Adam(learning_rate=0.0001, clipnorm=1., clipvalue=0.5, weight_decay=1e-7)
    model.compile(loss="binary_crossentropy",optimizer=adam_optimizer,metrics=['accuracy','f1_score'])
    model.summary()
    return model


In [27]:
# def build_model(time_series,num_of_features):
#     K.clear_session()
#     input_ = Input(shape=(time_series,num_of_features), name='Input')

#     encoder = GRU(256, return_state=True, return_sequences=True, name='Encoder')
#     encoder_outputs, state_h = encoder(input_)

#     output = Flatten(name='Flatten')(encoder_outputs)
#     output = Dense(128, activation='relu')(output)
#     output = Dense(1, activation='sigmoid')(output)

#     model = Model(inputs = input_ , outputs = output)
#     adam_optimizer = Adam(learning_rate=0.0001)
#     model.compile(loss='binary_crossentropy',optimizer=adam_optimizer,metrics=['accuracy','f1_score'])
#     model.summary()
#     return model

In [42]:
# Model Config
batch_size = 64
time_series = 20
input_shape = (time_series,196)
num_features=input_shape[1]

meta_df = pd.read_csv('data/meta.csv',index_col=0)
train_meta, test_meta = train_test_split(meta_df, test_size=0.2, random_state=4211)
# display(train_meta)
# display(test_meta)
train_generator = DataGenerator(train_meta,input_shape=input_shape, batch_size=batch_size)
val_generator = DataGenerator(test_meta,input_shape=input_shape, batch_size=batch_size)

KeyboardInterrupt: 

In [43]:

# Directory Config
neural_path = 'neural_network'
save_path_dir='attention_t20_fullFeatures_encode_decode'

if not os.path.exists(f'{neural_path}/{save_path_dir}'):
    os.makedirs(f'{neural_path}/{save_path_dir}')
    
checkpoint = ModelCheckpoint(
    f'{neural_path}/{save_path_dir}/saved_model.keras',
    verbose=1,
    monitor='val_f1_score',
    save_weights_only=False,
    save_best_only=True,
    mode='auto')

earlystop = EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    patience=20,
    verbose=0,
    mode="max",
    baseline=None,
    restore_best_weights=True
)

model = build_model(time_series,num_features)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)            │ (None, 20, 196)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 20, 256)           │          50,432 │ Input[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Encoder (GRU)                 │ [(None, 20, 256), (None,  │         394,752 │ dense[0][0]                │
│                               │ 256)]                     │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Attention (Attention)         │ (None, 20, 256)           │               0 │ Encoder[0][0],             │
│                               │                           │                 │ Encoder[0][1]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Concatenate (Concatenate)     │ (None, 20, 512)           │               0 │ Attention[0][0],           │
│                               │                           │                 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Decoder (GRU)                 │ (None, 20, 256)           │         591,360 │ Concatenate[0][0],         │
│                               │                           │                 │ Encoder[0][1]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Flatten (Flatten)             │ (None, 5120)              │               0 │ Decoder[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 5120)              │               0 │ Flatten[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 5120)              │          20,480 │ dropout[0][0]              │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 256)               │       1,310,976 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 256)               │               0 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 256)               │           1,024 │ dropout_1[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 128)               │          32,896 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_2 (Dropout)           │ (None, 128)               │               0 │ dense_2[0][0]              │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 2,402,561 (9.17 MB)

 Trainable params: 2,391,553 (9.12 MB)

 Non-trainable params: 11,008 (43.00 KB)

In [44]:
history = model.fit(train_generator, batch_size=batch_size, epochs=2000, validation_data=val_generator, callbacks=[checkpoint, earlystop])
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_accuracy'])
# plt.plot(history.history['val_f1_score'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
print(f"Highest Val Accuracy: {max(history.history['val_accuracy'])}")
# print(f"Highest Val F1: {max(history.history['val_f1_score'])}")

Epoch 1/2000


ValueError: Exception encountered when calling Attention.call().

[1mDimension must be 2 but is 3 for '{{node functional_1_1/Attention_1/transpose}} = Transpose[T=DT_FLOAT, Tperm=DT_INT32](functional_1_1/Encoder_1/while:4, functional_1_1/Attention_1/transpose/perm)' with input shapes: [?,256], [3].[0m

Arguments received by Attention.call():
  • inputs=['tf.Tensor(shape=(None, 20, 256), dtype=float32)', 'tf.Tensor(shape=(None, 256), dtype=float32)']
  • mask=['None', 'None']
  • training=True
  • return_attention_scores=False
  • use_causal_mask=False